In [15]:
import pandas as pd
import os 
import numpy as np

ruta = os.path.abspath('../Datasets')

In [2]:
df_compra = pd.read_csv(ruta + '/Compra.csv', sep=',', encoding='utf-8')

In [3]:
#eliminando IdCompra
df_compra.drop(['IdCompra'],axis=1,inplace=True)
#filas duplicadas
df_compra.duplicated().sum()

0

## Detección de Outliers

In [5]:
Q1 = df_compra['Precio'].quantile(.25)
Q3 = df_compra['Precio'].quantile(.75)
IQR = Q3 -Q1
minimo = Q1 - 1.5 * IQR
maximo = Q3 + 1.5 * IQR

In [6]:
ubicacion_outliers = (df_compra.Precio < minimo) | (df_compra.Precio > maximo)
df_compra[ubicacion_outliers].sort_values('Precio')

,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
580,4/24/2015,2015,4,201504,42782,10,2691.85,7
5833,4/27/2018,2018,4,201804,42936,3,2692.87,14
8607,10/25/2019,2019,1,201901,42782,22,2693.43,2
10422,7/31/2020,2020,7,202007,42780,6,2694.01,6
730,5/29/2015,2015,5,201505,42780,12,2694.41,6
...,...,...,...,...,...,...,...,...
6805,10/26/2018,2018,1,201801,42773,2,577223.00,13
7414,2/22/2019,2019,2,201902,42773,13,580886.00,6
8763,11/29/2019,2019,1,201901,42779,17,688826.00,10
11272,11/27/2020,2020,11,202011,42779,7,707004.00,8


In [7]:
#analizando uno de los posibles outliers
df_compra[df_compra.IdProducto == 42782].head()

,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
130,1/30/2015,2015,1,201501,42782,6,2678.34,1
580,4/24/2015,2015,4,201504,42782,10,2691.85,7
732,5/29/2015,2015,5,201505,42782,3,2347.42,13
878,6/26/2015,2015,6,201506,42782,4,2422.17,3
1186,8/28/2015,2015,8,201508,42782,4,2031.08,13


Podemos observar que el supuesto valor que es un outlier para la columna Precio, es un valor normal para el tipo de producto: 42782.
Lo que nos hace concluir que la estrategia que estamos usando para detectar outliers no es la adecuada.
Ya que la empresa tiene diferentes tipos de productos, buscaremos los valores atipicos en función del IdProducto.

In [8]:
#numero de diferentes productos existentes en la cartera de la empresa
df_compra.IdProducto.nunique()

283

In [9]:
#Escogemos un IdProducto: 42773 y calculamos el rango intercuantilico
df_id = df_compra[df_compra.IdProducto == 42773]
Q1 = df_id['Precio'].quantile(.25)
Q3 = df_id['Precio'].quantile(.75)
IQR = Q3 -Q1
minimo = Q1 - 1.5 * IQR
maximo = Q3 + 1.5 * IQR

In [10]:
#capturamos las filas con outliers
ubicacion_outliers = (df_id.Precio < minimo) | (df_id.Precio > maximo)
df_id[ubicacion_outliers]

,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
6805,10/26/2018,2018,1,201801,42773,2,577223.0,13
7414,2/22/2019,2019,2,201902,42773,13,580886.0,6


In [11]:
#calculamos la media, descartando los outliers
outlier = list(df_id[ubicacion_outliers].Precio)
media = (df_id.Precio.sum() - sum(outlier)) / (len(df_id) - len(outlier))
media

6102.406875000001

In [12]:
df_compra.Precio.mean()

3613.6468125671317

Podemos concluir que nuestro metodo de capturar los otliers por IdProducto es más certero.
Donde para la imputación de outliers del IdProducto: 42773 usariamos el precio de : 6102.41
y no el precio de : 3613.65

Antes de imputar los outliers podemos tambien descartar de la media los valores nulos y así tener una media más precisa.

## Detección de Valores Nulos

In [13]:
#capturando los valores nulos por columna
def valor_nulo(e):
    diccionario = {}
    for column in e.columns:
        val_nul = e[column].isna().sum()
        diccionario[column] = int(val_nul)

    return diccionario  

In [14]:
nulo = valor_nulo(df_compra)
nulo = [[key, nulo[key]] for key in nulo.keys() if nulo[key] > 0]
nulo

[['Precio', 367]]

Vemos que solo existen valores nulos en la columna Precio. 
A continuación imputaremos esos valores nulos, considerando que el valor a llenar depende del IdProducto, a la vez este depende de variables como el proveedor elegido (IdProveedor), el año de la compra (Fecha_Año), la temporada del año o mes (Fecha_Mes)

In [16]:
#imputación de valores faltantes de columna precio por columnas: IdProducto, IdProveedor, Fecha_Año
for i in list(zip(*np.where(df_compra['Precio'].isna()))):
    ii = i[0]
    pr = tuple(df_compra.loc[ii,['IdProducto']])[0]
    po = tuple(df_compra.loc[ii,['IdProveedor']])[0]
    fa = tuple(df_compra.loc[ii,['Fecha_Año']])[0]
    media =df_compra.loc[(df_compra['IdProducto'] == pr) & 
                    (df_compra['IdProveedor'] == po) & 
                    (df_compra['Fecha_Año'] == fa)]
    df_compra.loc[ii, ['Precio']] = media['Precio'].mean()

In [17]:
nulo = valor_nulo(df_compra)
nulo = [[key, nulo[key]] for key in nulo.keys() if nulo[key] > 0]
nulo

[['Precio', 231]]

Observamos  que el numero de nulos ah disminuido, elegiremos diferentes combinaciones de relación para ir llenando los valos que estén vacíos

In [18]:
#imputación de valores faltantes de columna precio por columnas: IdProducto, Fecha_Mes, Fecha_Año
for i in list(zip(*np.where(df_compra['Precio'].isna()))):
    ii = i[0]
    pr = tuple(df_compra.loc[ii,['IdProducto']])[0]
    po = tuple(df_compra.loc[ii,['Fecha_Mes']])[0]
    fa = tuple(df_compra.loc[ii,['Fecha_Año']])[0]
    media =df_compra.loc[(df_compra['IdProducto'] == pr) & 
                    (df_compra['Fecha_Mes'] == po) & 
                    (df_compra['Fecha_Año'] == fa)]
    df_compra.loc[ii, ['Precio']] = media['Precio'].mean()

In [19]:
nulo = valor_nulo(df_compra)
nulo = [[key, nulo[key]] for key in nulo.keys() if nulo[key] > 0]
nulo

[['Precio', 178]]

In [20]:
#imputación de valores faltantes de columna precio por columnas: IdProducto, IdProveedor, Fecha_Año
for i in list(zip(*np.where(df_compra['Precio'].isna()))):
    ii = i[0]
    pr = tuple(df_compra.loc[ii,['IdProducto']])[0]
    po = tuple(df_compra.loc[ii,['IdProveedor']])[0]
    media =df_compra.loc[(df_compra['IdProducto'] == pr) & 
                    (df_compra['IdProveedor'] == po)]
    df_compra.loc[ii, ['Precio']] = media['Precio'].mean()

In [21]:
nulo = valor_nulo(df_compra)
nulo = [[key, nulo[key]] for key in nulo.keys() if nulo[key] > 0]
nulo

[['Precio', 16]]

In [22]:
#imputación de valores faltantes de columna precio por columnas: IdProducto, Fecha_Año
for i in list(zip(*np.where(df_compra['Precio'].isna()))):
    ii = i[0]
    pr = tuple(df_compra.loc[ii,['IdProducto']])[0]
    po = tuple(df_compra.loc[ii,['Fecha_Año']])[0]
    media =df_compra.loc[(df_compra['IdProducto'] == pr) & 
                    (df_compra['Fecha_Año'] == po)]
    df_compra.loc[ii, ['Precio']] = media['Precio'].mean()

In [23]:
nulo = valor_nulo(df_compra)
nulo = [[key, nulo[key]] for key in nulo.keys() if nulo[key] > 0]
nulo

[]

En conclusión, hemos imputado los valores nulos con diferentes niveles de relación con el Precio, de mayor a menor relación según el criterio que he visto más conveniente.

Al momento de hacer la imputación, no he considerado que dentro de la media también se incluyen los outliers, generando valores sesgados. Para ello es recomendable primero imputar los outliers y luego los valores nulos.

In [24]:
df_compra.head()

,Fecha,Fecha_Año,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
0,1/30/2015,2015,1,201501,42832,13,560.51,12
1,1/30/2015,2015,1,201501,42833,11,497.58,7
2,1/30/2015,2015,1,201501,42834,1,588.50,6
3,1/30/2015,2015,1,201501,42835,9,567.66,14
4,1/30/2015,2015,1,201501,42839,14,231.31,2


In [25]:
df_compra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11539 entries, 0 to 11538
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fecha          11539 non-null  object 
 1   Fecha_Año      11539 non-null  int64  
 2   Fecha_Mes      11539 non-null  int64  
 3   Fecha_Periodo  11539 non-null  int64  
 4   IdProducto     11539 non-null  int64  
 5   Cantidad       11539 non-null  int64  
 6   Precio         11539 non-null  float64
 7   IdProveedor    11539 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 721.3+ KB


Para el resto de las columnas (Fecha, Fecha_Año, Fecha_Mes, Fecha_Periodo, IdProducto, IdProveedor), no se puede usar los metodos de imputación descritos arriba. Ya que su naturaleza es diferente.

Usaremos entonces dichos metodos solo para la columna Precio y cantidad.